In [ ]:
import pandas as pd
import datetime
import numpy as np
import re
import os
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from collections import defaultdict
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [ ]:
#To display all the columns and rows
pd.set_option('display.max_columns', 1000);
pd.set_option('display.max_rows', 1000);
pd.set_option('display.width', 1000);
#To ignore warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#To supress scientific notations
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
#Loading required data
d_icd_diagnoses = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Data/MIMIC/d_icd_diagnoses.csv")
diagnoses_icd = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Data/MIMIC/diagnoses_icd.csv")
patients = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Data/MIMIC/patients.csv")
admissions = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Data/MIMIC/admissions.csv")
orphanomen = pd.read_excel("/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Orphanet_Data/Nomenclature.xlsx")
orphadata = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vTecDAW5e7ZChOl597qtgkpwxyNeWlqJwvUtbE4gJTnDrAPapUuuyq4fi1XSTnzSDAQ5SECFl1E2H5r/pub?output=csv", header=1)

## Cleaning the orphanet rare diseases classification files into dataframes

In [ ]:
#Generating dataframes of all different categories of rare diseases along with their orphacode
# Set your path
xml_folder_path = '/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Orphanet_Data/Classifications'

# Dictionary to store DataFrames by file name
dataframes = {}

# Recursive extraction function
def extract_disorders(node):
    orphacodes = []
    disease_names = []

    for classification_node in node.findall(".//ClassificationNode"):
        disorder = classification_node.find("Disorder")
        if disorder is not None:
            disorder_type = disorder.find("DisorderType/Name")
            if disorder_type is not None and disorder_type.text == "Disease":
                orpha_code = disorder.find("OrphaCode")
                name = disorder.find("Name")
                if orpha_code is not None and name is not None:
                    orphacodes.append(orpha_code.text)
                    disease_names.append(name.text)

    return pd.DataFrame({
        'OrphaCode': orphacodes,
        'DiseaseName': disease_names
    })

# Loop through all XML files
for filename in os.listdir(xml_folder_path):
    if filename.endswith(".xml"):
        file_path = os.path.join(xml_folder_path, filename)
        tree = ET.parse(file_path)
        root = tree.getroot()
        df = extract_disorders(root)

        # Save using filename without extension as the key
        file_key = os.path.splitext(filename)[0]
        dataframes[file_key] = df

# Create separate DataFrames with dynamic variable names
for name, df in dataframes.items():
    var_name = f"{name}_df"
    globals()[var_name] = df

# Renaming the files into easier names

# Loop through all variables in the global scope
for var_name, var_value in list(globals().items()):
    if (var_name.startswith("ORPHAclassification") and
        var_name.endswith("_en_df") and
        isinstance(var_value, pd.DataFrame)):

        # Strip prefix and suffix
        new_name = var_name.removeprefix("ORPHAclassification").removesuffix("_en_df") + "_df"

        # Assign to new variable name
        globals()[new_name] = var_value

        # (Optional) delete the old variable
        del globals()[var_name]


for var_name, var_value in list(globals().items()):
    if (
        var_name.startswith("_") and
        var_name.endswith("_df") and
        isinstance(var_value, pd.DataFrame)
    ):
        # Extract the part starting from 'rare_' up to before '_en_df'
        start_index = var_name.find("rare_")
        end_index = var_name.rfind("_df")

        if start_index != -1:
            cleaned = var_name[start_index:end_index]
            new_name = f"{cleaned}_df"
            globals()[new_name] = var_value

            # Optional: delete original variable
            del globals()[var_name]

            print(f"Renamed: {var_name} → {new_name}")

Renamed: _152_rare_gastroenterological_diseases_df → rare_gastroenterological_diseases_df
Renamed: _197_rare_odontological_diseases_df → rare_odontological_diseases_df
Renamed: _212_rare_allergic_disease_df → rare_allergic_disease_df
Renamed: _146_rare_cardiac_diseases_df → rare_cardiac_diseases_df
Renamed: _231_rare_systemic_and_rheumatological_diseases_of_childhood_df → rare_systemic_and_rheumatological_diseases_of_childhood_df
Renamed: _185_rare_urogenital_diseases_df → rare_urogenital_diseases_df
Renamed: _183_rare_hepatic_diseases_df → rare_hepatic_diseases_df
Renamed: _195_rare_immunological_diseases_df → rare_immunological_diseases_df
Renamed: _156_rare_genetic_diseases_df → rare_genetic_diseases_df
Renamed: _186_rare_surgical_thoracic_diseases_df → rare_surgical_thoracic_diseases_df
Renamed: _193_rare_endocrine_diseases_df → rare_endocrine_diseases_df
Renamed: _182_rare_abdominal_surgical_diseases_df → rare_abdominal_surgical_diseases_df
Renamed: _209_rare_surgical_maxillo-faci

## Cleaning and combining orphanet disorders and mimic diagnosis

In [ ]:
# Cleaning orphadata (this is what I have extracted from the xml file by converting to excel)
# Dropping columns that are not required
orphadata.drop(columns=['DisorderListStatus','DisorderSource'], inplace=True)
# Removing the duplicates from the dataset
orphadata.drop_duplicates(subset='DisorderCode', inplace=True)
# Renaming the column
orphadata.rename(columns={'DisorderCode': 'ICDcodes'}, inplace=True)
# Resetting index
orphadata.reset_index(drop=True, inplace=True)

#Cleaning orphanomen (this is their nomenclature file which is provided in orphanet link)
# Dropping the Synonyms column as it is not required
orphanomen.drop(columns=['Synonyms'], inplace=True)
# Removing the duplicates from the dataset
orphanomen.drop_duplicates(subset='ICDcodes', inplace=True)
# Renaming the column
orphanomen.rename(columns={'PreferredTerm': 'OrphaName'}, inplace=True)
# Resetting index
orphanomen.reset_index(drop=True, inplace=True)

In [ ]:
orphadata.head()

,ICDcodes,DisorderName
0,Q77.3,Multiple epiphyseal dysplasia-macrocephaly-fac...
1,NaN,Multiple epiphyseal dysplasia-macrocephaly-fac...
2,G93.8,Alexander disease
3,E77.1,Alpha-mannosidosis
4,Q78.5,"Metaphyseal chondrodysplasia, Kaitila type"


In [ ]:
orphanomen.head()

,ORPHAcode,OrphaName,ICDcodes
0,166024,Multiple epiphyseal dysplasia-macrocephaly-fac...,Q77.3
1,166024,Multiple epiphyseal dysplasia-macrocephaly-fac...,NaN
2,58,Alexander disease,G93.8
3,61,Alpha-mannosidosis,E77.1
4,166038,"Metaphyseal chondrodysplasia, Kaitila type",Q78.5


In [ ]:
#Getting the final table for disease name, icd code and their orphacodes
#Merging the two tables on icd codes
disorders = pd.merge(orphadata, orphanomen, on='ICDcodes', how='left')
#converting orphacodes into integers
disorders['ORPHAcode'] = disorders['ORPHAcode'].astype('Int64')
#Drop Orphaname, just keeping DisorderName for the name of the disease
disorders.drop(columns=['OrphaName'], inplace=True)

In [ ]:
disorders.head(16)

,ICDcodes,DisorderName,ORPHAcode
0,Q77.3,Multiple epiphyseal dysplasia-macrocephaly-fac...,166024
1,NaN,Multiple epiphyseal dysplasia-macrocephaly-fac...,166024
2,G93.8,Alexander disease,58
3,E77.1,Alpha-mannosidosis,61
4,Q78.5,"Metaphyseal chondrodysplasia, Kaitila type",166038
5,Q87.8,Brachydactyly-short stature-retinitis pigmento...,166035
6,E75.2,Multiple sulfatase deficiency,585
7,Q04.3,Pontocerebellar hypoplasia type 4,166063
8,D68.0,Von Willebrand disease type 1,903
9,E72.0,Cystinosis,213


In [ ]:
# Creating a comparable column of icd 10 codes that can be used for mimic dataset mapping
#Define regex pattern for ICD-10 codes like Q77.3
pattern = r'^[A-Z][0-9]{2}\.[0-9]$'
#Find rows where ICDcodes doesn't match the pattern
other_codes = disorders[~disorders['ICDcodes'].astype(str).str.match(pattern, na=False)]
#function to clean the icd codes from Q77.3 to Q773 (as this is the format in MIMIC data)
def clean_icd(code):
    if pd.isna(code):
        return code
    code = str(code).strip()

    # Remove trailing non-alphanumeric characters like *, +, etc.
    code = re.sub(r'[^\w\.]+$', '', code)

    # Match ICD-10 format like Q77.3
    match = re.match(r'^([A-Z][0-9]{2})\.([0-9])$', code)
    if match:
        return match.group(1) + match.group(2)  # Remove dot, e.g., Q773
    else:
        return code  # Leave unchanged
# Apply to the dataframe
disorders['MIMICICDcode'] = disorders['ICDcodes'].apply(clean_icd)

In [ ]:
disorders.head(9)

,ICDcodes,DisorderName,ORPHAcode,MIMICICDcode
0,Q77.3,Multiple epiphyseal dysplasia-macrocephaly-fac...,166024,Q773
1,NaN,Multiple epiphyseal dysplasia-macrocephaly-fac...,166024,NaN
2,G93.8,Alexander disease,58,G938
3,E77.1,Alpha-mannosidosis,61,E771
4,Q78.5,"Metaphyseal chondrodysplasia, Kaitila type",166038,Q785
5,Q87.8,Brachydactyly-short stature-retinitis pigmento...,166035,Q878
6,E75.2,Multiple sulfatase deficiency,585,E752
7,Q04.3,Pontocerebellar hypoplasia type 4,166063,Q043
8,D68.0,Von Willebrand disease type 1,903,D680


In [ ]:
# Cleaning the table and merging with MIMIC data
#Rename the columns
disorders = disorders.rename(columns={'ORPHAcode': 'Orphacode','DisorderName': 'Disorder_Name', 'ICDcodes': 'ICD_10_Codes','MIMICICDcode': 'icd_code'})
#Reorder the columns
disorders = disorders[['Orphacode', 'Disorder_Name', 'ICD_10_Codes', 'icd_code']]
#Using only icd-10 codes
d_icd_diagnoses = d_icd_diagnoses[d_icd_diagnoses['icd_version']==10]
#Merging Orphanet disease naming data with MIMIC IV diagnoses table
disorders = pd.merge(disorders, d_icd_diagnoses, on='icd_code', how='inner')
#dropping icd_version as it is not required
disorders.drop(columns=['icd_version'], inplace=True)
#renaming the columns
disorders = disorders.rename(columns={'long_title': 'MIMIC_Disorder'})
#making orphacode a string so that it can be used for further comparisons
disorders['Orphacode'] = disorders['Orphacode'].astype(str)

In [ ]:
# Create a mapping from Orphanet files to a list of categories
orphacode_to_categories = defaultdict(set)

for var_name, df in list(globals().items()):
    if var_name.startswith("rare_") and isinstance(df, pd.DataFrame):
        if 'OrphaCode' in df.columns:
            for code in df['OrphaCode'].astype(str).unique():
                orphacode_to_categories[code].add(var_name.replace("_df", ""))
# Assign comma-separated list of categories to each row in disorders
disorders['Classification'] = disorders['Orphacode'].astype(str).map(
    lambda code: ", ".join(sorted(orphacode_to_categories.get(code, []))) if code in orphacode_to_categories else "Uncategorised"
)

In [ ]:
disorders.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2078 entries, 0 to 2077
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Orphacode       2078 non-null   object
 1   Disorder_Name   2078 non-null   object
 2   ICD_10_Codes    2078 non-null   object
 3   icd_code        2078 non-null   object
 4   MIMIC_Disorder  2078 non-null   object
 5   Classification  2078 non-null   object
dtypes: object(6)
memory usage: 97.5+ KB


In [ ]:
rare_skin_diseases_df.shape

(735, 2)

In [ ]:
#Getting rows of rare skin diseases only as that is our focus
disorders['Orphacode'] = disorders['Orphacode'].astype(str)
rare_skin_diseases_df['OrphaCode'] = rare_skin_diseases_df['OrphaCode'].astype(str)

# Merge the two dataframes on Orphacode
disorders_rare_skin = disorders[disorders['Orphacode'].isin(rare_skin_diseases_df['OrphaCode'])]
disorders_non_rare_skin = disorders[~disorders['Orphacode'].isin(rare_skin_diseases_df['OrphaCode'])]

## Selecting accurate diseases of various classes

### 1. Rare Skin Diseases - Positive Class

In [ ]:
!pip install rapidfuzz

In [ ]:
from rapidfuzz import fuzz

# Compute similarity score between Disorder_Name and MIMIC_Disorder
disorders_rare_skin['similarity'] = disorders_rare_skin.apply(
    lambda row: fuzz.token_sort_ratio(str(row['Disorder_Name']), str(row['MIMIC_Disorder'])),
    axis=1
)

# Flag similar rows based on threshold (e.g., 80%)
disorders_rare_skin['included'] = disorders_rare_skin['similarity'] >= 50

/tmp/ipython-input-86-56587904.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disorders_rare_skin['similarity'] = disorders_rare_skin.apply(
/tmp/ipython-input-86-56587904.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disorders_rare_skin['included'] = disorders_rare_skin['similarity'] >= 50


In [ ]:
#This is verified it is okay
disorders_rare_skin[disorders_rare_skin['included'] == True]

,Orphacode,Disorder_Name,ICD_10_Codes,icd_code,MIMIC_Disorder,Classification,similarity,included
34,461,Recessive X-linked ichthyosis,Q80.1,Q801,X-linked ichthyosis,"rare_genetic_diseases, rare_ophthalmic_disease...",79.167,True
54,163931,Acrodermatitis continua of Hallopeau,L40.2,L402,Acrodermatitis continua,"rare_genetic_diseases, rare_skin_diseases",77.966,True
55,163927,Pustulosis palmaris et plantaris,L40.3,L403,Pustulosis palmaris et plantaris,"rare_genetic_diseases, rare_skin_diseases",100.000,True
96,580,Mucopolysaccharidosis type 2,E76.1,E761,"Mucopolysaccharidosis, type II","rare_bone_diseases, rare_cardiac_diseases, rar...",93.103,True
100,163525,Subacute cutaneous lupus erythematosus,L93.1,L931,Subacute cutaneous lupus erythematosus,rare_skin_diseases,100.000,True
102,906,Wiskott-Aldrich syndrome,D82.0,D820,Wiskott-Aldrich syndrome,"rare_genetic_diseases, rare_hematological_dise...",100.000,True
120,636,Neurofibromatosis type 1,Q85.0,Q850,Neurofibromatosis (nonmalignant),"rare_bone_diseases, rare_developmental_anomali...",64.286,True
128,337,Fibrodysplasia ossificans progressiva,M61.1,M611,Myositis ossificans progressiva,"rare_bone_diseases, rare_developmental_anomali...",76.471,True
154,774,Hereditary hemorrhagic telangiectasia,I78.0,I780,Hereditary hemorrhagic telangiectasia,"rare_circulatory_system_diseases, rare_develop...",100.000,True
182,313,Lamellar ichthyosis,Q80.2,Q802,Lamellar ichthyosis,rare_developmental_anomalies_during_embryogene...,100.000,True


In [ ]:
#Manually setting the value as False, because these are ambiguous cases
disorders_rare_skin.loc[disorders_rare_skin['Orphacode'].isin(['178307', '140933']), 'included']=False

In [ ]:
disorders_rare_skin[disorders_rare_skin['Orphacode'] == '178307']

,Orphacode,Disorder_Name,ICD_10_Codes,icd_code,MIMIC_Disorder,Classification,similarity,included
373,178307,Reticulate acropigmentation of Kitamura,L81.8,L818,Other specified disorders of pigmentation,"rare_genetic_diseases, rare_skin_diseases",52.500,False


In [ ]:
#Checking all the False cases
disorders_rare_skin[(disorders_rare_skin['included'] == False) & (disorders_rare_skin['similarity'] > 40)]

,Orphacode,Disorder_Name,ICD_10_Codes,icd_code,MIMIC_Disorder,Classification,similarity,included
248,168606,Seborrhea-like dermatitis with psoriasiform el...,L21.8,L218,Other seborrheic dermatitis,"rare_genetic_diseases, rare_skin_diseases",48.101,False
373,178307,Reticulate acropigmentation of Kitamura,L81.8,L818,Other specified disorders of pigmentation,"rare_genetic_diseases, rare_skin_diseases",52.500,False
480,199276,Familial multiple lipomatosis,E88.2,E882,"Lipomatosis, not elsewhere classified","rare_genetic_diseases, rare_skin_diseases",45.455,False
553,2314,Autosomal dominant hyper-IgE syndrome due to S...,D82.4,D824,Hyperimmunoglobulin E [IgE] syndrome,"rare_genetic_diseases, rare_immunological_dise...",41.237,False
623,37,Acrodermatitis enteropathica,E83.2,E832,Disorders of zinc metabolism,rare_developmental_anomalies_during_embryogene...,42.857,False
635,1116,Aplasia cutis congenita-intestinal lymphangiec...,Q84.8,Q848,Other specified congenital malformations of in...,"rare_circulatory_system_diseases, rare_develop...",45.614,False
641,2222,"Congenital generalized hypertrichosis, Ambras ...",Q84.2,Q842,Other congenital malformations of hair,rare_developmental_anomalies_during_embryogene...,47.727,False
714,140933,Linear atrophoderma of Moulin,L90.8,L908,Other atrophic disorders of skin,"rare_genetic_diseases, rare_skin_diseases",52.459,False
729,1867,"Hereditary bullous dystrophy, macular type",Q81.8,Q818,Other epidermolysis bullosa,"rare_genetic_diseases, rare_neurological_disea...",43.478,False
753,2387,Leukonychia totalis,Q84.4,Q844,Congenital leukonychia,"rare_genetic_diseases, rare_skin_diseases",48.780,False


In [ ]:
#Checking the left over records
disorders_rare_skin[disorders_rare_skin['included'] == False]

,Orphacode,Disorder_Name,ICD_10_Codes,icd_code,MIMIC_Disorder,Classification,similarity,included
5,585,Multiple sulfatase deficiency,E75.2,E752,Other sphingolipidosis,"rare_bone_diseases, rare_developmental_anomali...",27.451,False
13,166113,Bazex syndrome,L44.8,L448,Other specified papulosquamous disorders,"rare_genetic_diseases, rare_neoplastic_disease...",29.630,False
20,166286,Porokeratotic eccrine ostial and dermal duct n...,Q82.5,Q825,Congenital non-neoplastic nevus,"rare_genetic_diseases, rare_neoplastic_disease...",39.506,False
68,100,Ataxia-telangiectasia,G11.3,G113,Cerebellar ataxia with defective DNA repair,rare_developmental_anomalies_during_embryogene...,37.500,False
119,163634,Maffucci syndrome,Q78.4,Q784,Enchondromatosis,"rare_bone_diseases, rare_circulatory_system_di...",36.364,False
123,379,Chronic granulomatous disease,D71,D71,Functional disorders of polymorphonuclear neut...,"rare_gastroenterological_diseases, rare_geneti...",39.024,False
136,167,Waardenburg syndrome type 2,E70.3,E703,Albinism,"rare_genetic_diseases, rare_hematological_dise...",17.143,False
165,562,McCune-Albright syndrome,Q78.1,Q781,Polyostotic fibrous dysplasia,"rare_bone_diseases, rare_developmental_anomali...",22.642,False
214,218,Darier disease,Q82.8,Q828,Other specified congenital malformations of skin,"rare_genetic_diseases, rare_skin_diseases",25.806,False
230,56,Alkaptonuria,E70.2,E702,Disorders of tyrosine metabolism,"rare_genetic_diseases, rare_inborn_errors_of_m...",18.182,False


In [ ]:
#Manually setting the value as True, because these are similar diseases different names
disorders_rare_skin.loc[disorders_rare_skin['Orphacode'].isin(['645849', '313846', '618', '2314', '37', '199276', '636955', '168606', '346',
                                                               '345', '312', '2222', '2387', '2481', '447961', '499', '97352', '528', '100',
                                                               '46487', '168', '228272', '3243', '901', '199279', '65748', '314', '300359',
                                                               '79100', '90291', '199251', '562', '79404', '1366', '1410', '2309']), 'included']=True

In [ ]:
disorders_rare_skin.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 193 entries, 5 to 2070
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Orphacode       193 non-null    object 
 1   Disorder_Name   193 non-null    object 
 2   ICD_10_Codes    193 non-null    object 
 3   icd_code        193 non-null    object 
 4   MIMIC_Disorder  193 non-null    object 
 5   Classification  193 non-null    object 
 6   similarity      193 non-null    float64
 7   included        193 non-null    bool   
dtypes: bool(1), float64(1), object(6)
memory usage: 12.3+ KB


In [ ]:
#Check MIMIC_Disorder records that start with 'Other' (make all of these maybe)
#Convert 'included' column to string type
disorders_rare_skin['included'] = disorders_rare_skin['included'].astype(str)
#Set 'included' to "Maybe" if MIMIC_Disorder starts with "Other"
disorders_rare_skin.loc[disorders_rare_skin['MIMIC_Disorder'].str.startswith('Other', na=False),'included'] = 'Maybe'
#Setting Maybe to more records that have a rare condition in the broader ICD code and have no other specific ICD code for themselves
disorders_rare_skin.loc[disorders_rare_skin['Orphacode'].isin(['379', '1221', '293173', '330058', '79153', '254492', '95429',
                                                               '167635', '163634', '166286']), 'included']= 'Maybe'

/tmp/ipython-input-94-3254202968.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disorders_rare_skin['included'] = disorders_rare_skin['included'].astype(str)


In [ ]:
#Checking for maybe disorders
disorders_rare_skin[disorders_rare_skin['included'] == 'False']

,Orphacode,Disorder_Name,ICD_10_Codes,icd_code,MIMIC_Disorder,Classification,similarity,included
136,167,Waardenburg syndrome type 2,E70.3,E703,Albinism,"rare_genetic_diseases, rare_hematological_dise...",17.143,False
230,56,Alkaptonuria,E70.2,E702,Disorders of tyrosine metabolism,"rare_genetic_diseases, rare_inborn_errors_of_m...",18.182,False
639,139414,Congenital panfollicular nevus,I78.1,I781,"Nevus, non-neoplastic","rare_neoplastic_diseases, rare_skin_diseases",39.216,False
1236,293812,Fixed drug eruption,L25.1,L251,Unspecified contact dermatitis due to drugs in...,rare_skin_diseases,28.916,False
1491,302,Inherited epidermodysplasia verruciformis,B07,B07,Viral warts,"rare_genetic_diseases, rare_immunological_dise...",23.077,False
1870,85453,X-linked reticulate pigmentary disorder,E85.0+,E850,Non-neuropathic heredofamilial amyloidosis,"rare_genetic_diseases, rare_immunological_dise...",34.568,False


In [ ]:
# Replace B07 → B078 and set included = 'True'
disorders_rare_skin.loc[disorders_rare_skin['icd_code'] == 'B07', ['icd_code', 'included']] = ['B078', 'True']

# Replace E702 → E7029 and set included = 'True'
disorders_rare_skin.loc[disorders_rare_skin['icd_code'] == 'E702', ['icd_code', 'included']] = ['E7029', 'True']

In [ ]:
#Exclude these from the data
disorders_rare_skin[disorders_rare_skin['included'] == 'False']

,Orphacode,Disorder_Name,ICD_10_Codes,icd_code,MIMIC_Disorder,Classification,similarity,included
136,167,Waardenburg syndrome type 2,E70.3,E703,Albinism,"rare_genetic_diseases, rare_hematological_dise...",17.143,False
639,139414,Congenital panfollicular nevus,I78.1,I781,"Nevus, non-neoplastic","rare_neoplastic_diseases, rare_skin_diseases",39.216,False
1236,293812,Fixed drug eruption,L25.1,L251,Unspecified contact dermatitis due to drugs in...,rare_skin_diseases,28.916,False
1870,85453,X-linked reticulate pigmentary disorder,E85.0+,E850,Non-neuropathic heredofamilial amyloidosis,"rare_genetic_diseases, rare_immunological_dise...",34.568,False


In [ ]:
#dropping similarity column as it is not required
disorders_rare_skin.drop(columns=['similarity'], inplace=True)

/tmp/ipython-input-98-3439507688.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disorders_rare_skin.drop(columns=['similarity'], inplace=True)


In [ ]:
#Write the data to the folder
disorders_rare_skin.to_excel("/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Data/disorders_rare_skin.xlsx", index=False, header = True)

###2. Common Skin Diseases - Negative Class

Skuhala T, Trkulja V, Rimac M, Dragobratović A, Desnica B. Analysis of Types of Skin Lesions and Diseases in Everyday Infectious Disease Practice-How Experienced Are We? Life (Basel). 2022 Jun 29;12(7):978. doi: 10.3390/life12070978. PMID: 35888068; PMCID: PMC9319552.

Hay, R.J., Johns, N.E., Williams, H.C., Bolliger, I.W., Dellavalle, R.P., Margolis, D.J., Marks, R., Naldi, L., Weinstock, M.A., Wulf, S.K. and Michaud, C., 2014. The global burden of skin disease in 2010: an analysis of the prevalence and impact of skin conditions. Journal of investigative dermatology, 134(6), pp.1527-1534.

In [ ]:
common_skin_disease_icd_codes = ['B351', 'B352', 'B353', 'B354', 'B355', 'B356', 'B359',
                                 'L089','L200', 'L209', 'L219', 'L230', 'L231', 'L232', 'L233', 'L235', 'L237', 'L2389', 'L239',
                                 'L249', 'L259', 'L280', 'L281',
                                 'L300', 'L309',
                                 'L400', 'L404', 'L405', 'L409',
                                 'L500', 'L501', 'L503', 'L505', 'L506', 'L508', 'L509', 'L550', 'L551', 'L559', 'L570',
                                 'L700', 'L701', 'L702', 'L703', 'L704', 'L705', 'L708', 'L709', 'L710', 'L719', 'L720', 'L723',
                                 'L820', 'L821', 'L84', 'L853',
                                 'L910', 'R21', 'R22']

#Filter rows where icd_code is in the list, then select both 'icd_code' and 'long_title'
disorders_common_skin = d_icd_diagnoses[d_icd_diagnoses['icd_code'].isin(common_skin_disease_icd_codes)][['icd_code', 'long_title']].copy()
#Add other columns
disorders_common_skin['Classification'] = 'common_skin_diseases'
disorders_common_skin['included'] = 'True'
#Rename long_title to MIMIC_Disorder
disorders_common_skin = disorders_common_skin.rename(columns={'long_title': 'MIMIC_Disorder'})

In [ ]:
disorders_common_skin

,icd_code,MIMIC_Disorder,Classification,included
13141,B351,Tinea unguium,common_skin_diseases,True
13142,B352,Tinea manuum,common_skin_diseases,True
13143,B353,Tinea pedis,common_skin_diseases,True
13144,B354,Tinea corporis,common_skin_diseases,True
13145,B355,Tinea imbricata,common_skin_diseases,True
13146,B356,Tinea cruris,common_skin_diseases,True
13148,B359,"Dermatophytosis, unspecified",common_skin_diseases,True
27868,L089,Local infection of the skin and subcutaneous t...,common_skin_diseases,True
27902,L200,Besnier's prurigo,common_skin_diseases,True
27909,L209,"Atopic dermatitis, unspecified",common_skin_diseases,True


In [ ]:
disorders_common_skin.shape

(59, 4)

In [ ]:
#Write the data to the folder
disorders_common_skin.to_excel("/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Data/disorders_common_skin.xlsx", index=False, header = True)

### 3. Rare Diseases - Negative Class

In [ ]:
#Creating a table for just rare diseases (these don't include rare skin diseases)
disorders_rare = disorders_non_rare_skin[disorders_non_rare_skin['Classification'] != 'Uncategorised'].copy()
#Adding other column
disorders_rare['included'] = 'True'

In [ ]:
# Display rows where ICD_10_Codes and icd_code are not equal
disorders_rare_copy = disorders_rare.copy()

# Define normalization function
def normalize_icd(code):
    if pd.isna(code):
        return ''
    return code.replace('.', '').replace('+', '').strip().upper()

# Apply normalization to the copy
disorders_rare_copy['ICD_10_Codes_norm'] = disorders_rare_copy['ICD_10_Codes'].apply(normalize_icd)
disorders_rare_copy['icd_code_norm'] = disorders_rare_copy['icd_code'].apply(normalize_icd)

# Filter rows where normalized ICD codes differ
mismatched_icd_rows = disorders_rare_copy[
    disorders_rare_copy['ICD_10_Codes_norm'] != disorders_rare_copy['icd_code_norm']
]
mismatched_icd_rows

,Orphacode,Disorder_Name,ICD_10_Codes,icd_code,MIMIC_Disorder,Classification,included,ICD_10_Codes_norm,icd_code_norm
476,3452,Whipple disease,M14.8*,M148,Arthropathies in other specified diseases clas...,"rare_gastroenterological_diseases, rare_geneti...",True,M148*,M148
1192,439254,ABetaA21G amyloidosis,I68.0*,I680,Cerebral amyloid angiopathy,"rare_genetic_diseases, rare_neurological_disea...",True,I680*,I680
1728,33475,Meningococcal meningitis,G01*,G01,Meningitis in bacterial diseases classified el...,rare_neurological_diseases,True,G01*,G01
2015,645822,Primary bone and joint tuberculosis,"A18.0,",A180,Tuberculosis of bones and joints,"rare_bone_diseases, rare_infectious_diseases",True,"A180,",A180
2068,641396,Central nervous system tuberculosis,G07*,G07,Intracranial and intraspinal abscess and granu...,"rare_infectious_diseases, rare_neurological_di...",True,G07*,G07


In [ ]:
# Get the list of icd_codes from mismatched_icd_rows
mismatched_codes = mismatched_icd_rows['icd_code'].unique()

# Update 'included' column in disorders_rare (as string)
disorders_rare.loc[disorders_rare['icd_code'].isin(mismatched_codes), 'included'] = 'False'

In [ ]:
#Write the data to the folder
disorders_rare.to_excel("/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Data/disorders_rare.xlsx", index=False, header = True)

### 4. Common Diseases - Negative Class

Cite the following resources:

The Healthcare Cost and Utilization Project (HCUP) regularly publishes statistics on the most common principal diagnoses for hospital stays. Top Diagnoses in adult inpatient admissions (2018–2022) include: Sepsis (A41),Heart failure (I50.x),Diabetes mellitus (E11.x),Pneumonia (J18.x),COPD (J44.x),Acute kidney injury (N17.x),Urinary tract infection (N39.0),Iron deficiency anemia (D50.x)

Healthcare Cost and Utilization Project (HCUP). Most Frequent Principal Diagnoses for Inpatient Stays. Agency for Healthcare Research and Quality (AHRQ). https://hcup-us.ahrq.gov/reports.jsp

Chronic conditions like hypertension, diabetes, heart failure, and COPD consistently rank among the top causes of DALYs and YLDs globally.

GBD 2019 Diseases and Injuries Collaborators. Global burden of 369 diseases and injuries in 204 countries and territories, 1990–2019: a systematic analysis. The Lancet. 2020;396(10258):1204–1222. https://doi.org/10.1016/S0140-6736(20)30925-9

Several studies analyzing MIMIC-III and MIMIC-IV confirm that these same diseases are common in critical care settings. Examples: Sepsis and AKI are among the top causes of ICU admission. Diabetes and hypertension are leading comorbidities.
A JMIR Cancer study reports ICU prevalence estimates from MIMIC‑III, including:
Hypertension (≈48%), diabetes mellitus (≈21%), congestive heart failure (≈27%), pneumonia (≈12%), anemia (≈14%), chronic pulmonary disease (~10%)
Though these statistics are from MIMIC‑III, they provide strong evidence that your chosen control diseases are among the most commonly coded conditions in critical care.

https://en.wikipedia.org/wiki/List_of_causes_of_death_by_rate

In [ ]:
common_disease_icd_codes = ['I10', 'E119', 'I5022', 'I5023', 'I5032', 'I5033', 'I509', 'J440', 'J441', 'J449', 'J189', 'N179', 'A4151', 'A4101', 'A419', 'D509']
#common_disease_icd_codes = ['E119','E785','K219','J449','D509','M791','J069','N390','F419','I509']

# Filter rows where icd_code is in the list, then select both 'icd_code' and 'long_title'
disorders_common = d_icd_diagnoses[
    d_icd_diagnoses['icd_code'].isin(common_disease_icd_codes)
][['icd_code', 'long_title']].copy()

# Add additional columns
disorders_common['Classification'] = 'common_diseases'
disorders_common['included'] = 'True'

# Rename 'long_title' to 'MIMIC_Disorder'
disorders_common = disorders_common.rename(columns={'long_title': 'MIMIC_Disorder'})

In [ ]:
disorders_common

,icd_code,MIMIC_Disorder,Classification,included
12537,A4101,Sepsis due to Methicillin susceptible Staphylo...,common_diseases,True
12545,A4151,Sepsis due to Escherichia coli [E. coli],common_diseases,True
12552,A419,"Sepsis, unspecified organism",common_diseases,True
15631,D509,"Iron deficiency anemia, unspecified",common_diseases,True
16575,E119,Type 2 diabetes mellitus without complications,common_diseases,True
24498,I10,Essential (primary) hypertension,common_diseases,True
24785,I5022,Chronic systolic (congestive) heart failure,common_diseases,True
24786,I5023,Acute on chronic systolic (congestive) heart f...,common_diseases,True
24790,I5032,Chronic diastolic (congestive) heart failure,common_diseases,True
24791,I5033,Acute on chronic diastolic (congestive) heart ...,common_diseases,True


In [ ]:
#Write the data to the folder
disorders_common.to_excel("/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Data/disorders_common.xlsx", index=False, header = True)

### 5. Uncategorised Diseases

In [ ]:
#Creating a table for just rare diseases (these don't include rare skin diseases)
disorders_unspecified = disorders_non_rare_skin[disorders_non_rare_skin['Classification'] == 'Uncategorised'].copy()
#Adding other column
disorders_unspecified['included'] = 'True'

In [ ]:
disorders_unspecified

,Orphacode,Disorder_Name,ICD_10_Codes,icd_code,MIMIC_Disorder,Classification,included
4,166035,Brachydactyly-short stature-retinitis pigmento...,Q87.8,Q878,Other specified congenital malformation syndro...,Uncategorised,True
6,166063,Pontocerebellar hypoplasia type 4,Q04.3,Q043,Other reduction deformities of brain,Uncategorised,True
10,166100,Autosomal dominant otospondylomegaepiphyseal d...,Q87.0,Q870,Congenital malformation syndromes predominantl...,Uncategorised,True
15,1031,Dentinogenesis imperfecta type 2,K00.5,K005,"Hereditary disturbances in tooth structure, no...",Uncategorised,True
35,<NA>,NON RARE IN EUROPE: Tourette syndrome,F95.2,F952,Tourette's disorder,Uncategorised,True
36,<NA>,NON RARE IN EUROPE: Ankylosing spondylitis,M45,M45,Ankylosing spondylitis,Uncategorised,True
37,881,Turner syndrome,Q96.0,Q960,"Karyotype 45, X",Uncategorised,True
38,881,Turner syndrome,Q96.1,Q961,"Karyotype 46, X iso (Xq)",Uncategorised,True
39,881,Turner syndrome,Q96.8,Q968,Other variants of Turner's syndrome,Uncategorised,True
40,881,Turner syndrome,Q96.9,Q969,"Turner's syndrome, unspecified",Uncategorised,True


In [ ]:
#Write the data to the folder
disorders_unspecified.to_excel("/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Data/disorders_unspecified.xlsx", index=False, header = True)

###Combining into a single dataset.

In [ ]:
# Rare skin disorders → class 1
disorders_rare_skin1 = disorders_rare_skin[['MIMIC_Disorder', 'icd_code', 'Classification', 'included']].copy()
disorders_rare_skin1['class'] = 1

# Common skin disorders
disorders_common_skin1 = disorders_common_skin[['MIMIC_Disorder', 'icd_code', 'Classification', 'included']].copy()
disorders_common_skin1['class'] = 2

# Rare diseases (non-skin)
disorders_rare1 = disorders_rare[['MIMIC_Disorder', 'icd_code', 'Classification', 'included']].copy()
disorders_rare1['class'] = 3

# Common diseases
disorders_common1 = disorders_common[['MIMIC_Disorder', 'icd_code', 'Classification', 'included']].copy()
disorders_common1['class'] = 4

# Unspecified/Uncategorized but included
disorders_unspecified1 = disorders_unspecified[['MIMIC_Disorder', 'icd_code', 'Classification', 'included']].copy()
disorders_unspecified1['class'] = 5

# Combine all 5 tables
all_disorders = pd.concat([disorders_rare_skin1,disorders_common_skin1,disorders_common1,disorders_rare1,disorders_unspecified1], ignore_index=True)

In [ ]:
all_disorders.shape

(2153, 5)

In [ ]:
all_disorders['class'].value_counts()

,count
class,
3,1297
5,588
1,193
2,59
4,16


## Filtering EHR data based on the selected diseases

In [ ]:
diagnoses_icd.shape

(4752265, 5)

In [ ]:
#Using only icd-10 codes
d_icd_diagnoses = d_icd_diagnoses[d_icd_diagnoses['icd_version']==10]
diagnoses_icd = diagnoses_icd[diagnoses_icd['icd_version']==10]

In [ ]:
diagnoses_icd.shape

(1981807, 5)

In [ ]:
#combining diagnoses tables for better usage
diagnoses = d_icd_diagnoses.merge(diagnoses_icd, on='icd_code', how='inner')
#cleaning the table by removing unnecessary columns
diagnoses.drop(columns=['icd_version_x','icd_version_y'], inplace=True)
#converting columns into int
diagnoses['subject_id'] = diagnoses['subject_id'].astype('Int64')
diagnoses['hadm_id'] = diagnoses['hadm_id'].astype('Int64')
diagnoses['seq_num'] = diagnoses['seq_num'].astype('Int64')

In [ ]:
diagnoses.shape

(1981807, 5)

In [ ]:
#Combining diagnoses with all_disorders and cleaning the resulting table
# Filter all_disorders to exclude records where included == 'False'
all_disorders_filtered = all_disorders[all_disorders['included'] != 'False'].copy()

# Perform inner merge with diagnoses
merged_diagnoses = diagnoses.merge(all_disorders_filtered,on='icd_code', how='inner')
# Dropping MIMIC_Disorder column as we have a duplicate in long_title
merged_diagnoses.drop(columns=['MIMIC_Disorder'], inplace=True)
# Checking the records
merged_diagnoses['class'].value_counts()

,count
class,
4,140008
5,70176
3,62610
2,4733
1,3177


In [ ]:
# Perform inner merge with patients
merged_patients = merged_diagnoses.merge(patients,on='subject_id', how='inner')

In [ ]:
merged_patients

,icd_code,long_title,subject_id,hadm_id,seq_num,Classification,included,class,gender,anchor_age,anchor_year,anchor_year_group,dod
0,A028,Other specified salmonella infections,18131667,25268955,9,rare_infectious_diseases,True,3,F,41,2185,2008 - 2010,NaN
1,A030,Shigellosis due to Shigella dysenteriae,16281124,26566783,2,rare_infectious_diseases,True,3,M,26,2162,2017 - 2019,NaN
2,A031,Shigellosis due to Shigella flexneri,10139228,27281102,1,rare_infectious_diseases,True,3,M,38,2123,2008 - 2010,NaN
3,A031,Shigellosis due to Shigella flexneri,11665789,23217190,1,rare_infectious_diseases,True,3,M,44,2140,2011 - 2013,NaN
4,A033,Shigellosis due to Shigella sonnei,16926894,25181412,1,rare_infectious_diseases,True,3,M,53,2127,2017 - 2019,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
280699,Y841,Kidney dialysis as the cause of abnormal react...,19530208,28622231,22,Uncategorised,True,5,F,52,2188,2008 - 2010,NaN
280700,Y841,Kidney dialysis as the cause of abnormal react...,19625808,23157744,6,Uncategorised,True,5,F,50,2178,2011 - 2013,2183-05-31
280701,Y841,Kidney dialysis as the cause of abnormal react...,19625808,24847963,11,Uncategorised,True,5,F,50,2178,2011 - 2013,2183-05-31
280702,Y841,Kidney dialysis as the cause of abnormal react...,19640899,20457378,18,Uncategorised,True,5,F,75,2185,2011 - 2013,2190-04-27


In [ ]:
#Merge merged_patients with admissions
from collections import defaultdict

# Merge merged_patients with admissions
merged_adm_pat = merged_patients.merge(admissions, on=['subject_id', 'hadm_id'], how='inner')

# Ensure datetime types
merged_adm_pat['admittime'] = pd.to_datetime(merged_adm_pat['admittime'])
merged_adm_pat['dischtime'] = pd.to_datetime(merged_adm_pat['dischtime'])

# Sort the merged dataframe
merged_adm_pat_sorted = merged_adm_pat.sort_values(by=['subject_id', 'admittime', 'class', 'seq_num'])

# 1. First Admission
merged_adm_pat_sorted['is_first_admission'] = (
    merged_adm_pat_sorted.groupby('subject_id')['admittime'].rank(method='first') == 1
).astype(int)

# 2. Number of follow-ups after first visit
adm_map = merged_adm_pat_sorted[['subject_id', 'hadm_id', 'admittime']].drop_duplicates()
adm_map = adm_map.sort_values(by=['subject_id', 'admittime'])

followup_count_map = defaultdict(int)
for subject_id, group in adm_map.groupby('subject_id'):
    adms = list(group.itertuples(index=False))
    n = len(adms)
    for i in range(n):
        followup_count_map[(adms[i].subject_id, adms[i].hadm_id)] = n - i - 1

merged_adm_pat_sorted['num_followups'] = merged_adm_pat_sorted.apply(
    lambda row: followup_count_map[(row['subject_id'], row['hadm_id'])], axis=1
)

# 3. Was this patient diagnosed for rare skin diseases in the future?
rare_skin_map = merged_adm_pat_sorted[merged_adm_pat_sorted['class'] == 1]\
    .groupby('subject_id')['admittime'].min().to_dict()

def has_future_rare_skin(row):
    future_rare_date = rare_skin_map.get(row['subject_id'], None)
    if future_rare_date and future_rare_date > row['admittime']:
        return 1
    return 0

merged_adm_pat_sorted['future_rare_skin'] = merged_adm_pat_sorted.apply(has_future_rare_skin, axis=1)

# 4. All ICD-10 codes for the patient for the particular hospitalisation
icd_combined = merged_adm_pat_sorted.groupby(['subject_id', 'hadm_id'])['icd_code']\
    .unique().apply(lambda codes: ','.join(codes)).reset_index(name='icd_code_list')
merged_adm_pat_sorted = merged_adm_pat_sorted.merge(icd_combined, on=['subject_id', 'hadm_id'], how='left')

# 5. Days to Next Admission (using adm_map which is already unique + sorted)

# Compute next admission time
adm_map['next_admittime'] = adm_map.groupby('subject_id')['admittime'].shift(-1)

# Compute days to next admission
# Merge dischtime into adm_map so we can subtract
adm_map = adm_map.merge(
    merged_adm_pat_sorted[['subject_id', 'hadm_id', 'dischtime']].drop_duplicates(),
    on=['subject_id', 'hadm_id'],
    how='left'
)

# Calculate days_to_next_admission
adm_map['days_to_next_admission'] = (
    (adm_map['next_admittime'] - adm_map['dischtime']).dt.days
)

# Merge back into full dataset
merged_adm_pat_sorted = merged_adm_pat_sorted.merge(
    adm_map[['subject_id', 'hadm_id', 'next_admittime', 'days_to_next_admission']],
    on=['subject_id', 'hadm_id'],
    how='left'
)

# 6. Time to Diagnosis (Number of admissions until rare skin diagnosis)
def get_time_to_diagnosis(group):
    group = group.sort_values(by='admittime').reset_index(drop=True)
    # Look for first row where class == 1
    if (group['class'] == 1).any():
        diagnosis_idx = group[group['class'] == 1].index[0]
        group['time_to_diagnosis'] = diagnosis_idx + 1  # Admission count is 1-indexed
    else:
        group['time_to_diagnosis'] = None  # Never diagnosed with rare skin disease
    return group

merged_adm_pat_sorted = merged_adm_pat_sorted.groupby('subject_id', group_keys=False).apply(get_time_to_diagnosis)

# Keep only first admission records if needed
merged_admissions = merged_adm_pat_sorted[merged_adm_pat_sorted['is_first_admission'] == 1].copy()


/tmp/ipython-input-125-4153819066.py:87: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_adm_pat_sorted = merged_adm_pat_sorted.groupby('subject_id', group_keys=False).apply(get_time_to_diagnosis)


In [ ]:
#BACKUP CODE DO NOT DELETE!!
#Merge merged_patients with admissions
#merged_adm_pat = merged_patients.merge(admissions, on=['subject_id', 'hadm_id'], how='inner')
#Sorting the merged dataframe by subject_id, admittime and class
#merged_adm_pat_sorted = merged_adm_pat.sort_values(by=['subject_id', 'admittime', 'class', 'seq_num'])

#Creating new columns as required
#1. First Admission
#merged_adm_pat_sorted['is_first_admission'] = (merged_adm_pat_sorted.groupby('subject_id')['admittime'].rank(method='first') == 1).astype(int)

#2. Number of follow ups after first visit
# Get all admissions per patient, sorted
#adm_map = merged_adm_pat_sorted[['subject_id', 'hadm_id', 'admittime']].drop_duplicates()
#adm_map = adm_map.sort_values(by=['subject_id', 'admittime'])

# Map each admission to how many future admissions follow it
#followup_count_map = defaultdict(int)

#for subject_id, group in adm_map.groupby('subject_id'):
#    adms = list(group.itertuples(index=False))
#    n = len(adms)
#    for i in range(n):
#        followup_count_map[(adms[i].subject_id, adms[i].hadm_id)] = n - i - 1

# Map back to the dataframe
#merged_adm_pat_sorted['num_followups'] = merged_adm_pat_sorted.apply(lambda row: followup_count_map[(row['subject_id'], row['hadm_id'])], axis=1)

#3. Was this patient diagnosed for rare skin diseases in the future?
#Getting the future rare skin disease admissions
#rare_skin_map = merged_adm_pat_sorted[merged_adm_pat_sorted['class'] == 1].groupby('subject_id')['admittime'].min().to_dict()# earliest rare skin dx per patient

#Define the indicator column
#def has_future_rare_skin(row):
#    future_rare_date = rare_skin_map.get(row['subject_id'], None)
#    if future_rare_date and future_rare_date > row['admittime']:
#        return 1
#    return 0

#merged_adm_pat_sorted['future_rare_skin'] = merged_adm_pat_sorted.apply(has_future_rare_skin, axis=1)

#4. All ICD-10 codes for the patient for the particular hospitalisation
#Combine all ICD codes for each admission
#icd_combined = merged_adm_pat_sorted.groupby(['subject_id', 'hadm_id'])['icd_code'].unique().apply(lambda codes: ','.join(codes)).reset_index(name='icd_code_list')
#Merge back to the original dataframe
#merged_adm_pat_sorted = merged_adm_pat_sorted.merge(icd_combined, on=['subject_id', 'hadm_id'], how='left')

#Keeping only first admission records
#merged_admissions = merged_adm_pat_sorted[merged_adm_pat_sorted['is_first_admission'] == 1].copy()

In [ ]:
merged_admissions[merged_admissions['class']==4]['icd_code'].value_counts()

,count
icd_code,
I10,14527
N179,3705
E119,2436
J449,1364
J189,1341
A419,1072
I5033,862
I5032,788
D509,656


In [ ]:
#Checking counts
merged_admissions['class'].value_counts()

,count
class,
4,29033
3,21987
2,2280
5,2202
1,1415


In [ ]:
#Checking for duplicates
merged_admissions[merged_admissions.duplicated('subject_id', keep=False)]

,icd_code,long_title,subject_id,hadm_id,seq_num,Classification,included,class,gender,anchor_age,anchor_year,anchor_year_group,dod,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,is_first_admission,num_followups,future_rare_skin,icd_code_list,next_admittime,days_to_next_admission,time_to_diagnosis


In [ ]:
merged_admissions.tail(7)

,icd_code,long_title,subject_id,hadm_id,seq_num,Classification,included,class,gender,anchor_age,anchor_year,anchor_year_group,dod,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,is_first_admission,num_followups,future_rare_skin,icd_code_list,next_admittime,days_to_next_admission,time_to_diagnosis
0,D531,"Other megaloblastic anemias, not elsewhere cla...",19997843,20277361,23,rare_developmental_anomalies_during_embryogene...,True,3,M,47,2120,2017 - 2019,NaN,2120-11-17 23:16:00,2120-11-28 15:40:00,NaN,EW EMER.,CLINIC REFERRAL,SKILLED NURSING FACILITY,Other,?,NaN,UNKNOWN,2120-11-17 21:25:00,2120-11-18 02:39:00,0,1,0,0,"D531,G252,I10",NaT,NaN,None
0,K743,Primary biliary cirrhosis,19997886,20793010,17,"rare_hepatic_diseases, rare_transplant-related...",True,3,M,67,2181,2011 - 2013,2186-12-11,2186-11-12 07:10:00,2186-12-10 20:35:00,NaN,EW EMER.,CLINIC REFERRAL,HOSPICE,Other,ENGLISH,WIDOWED,WHITE,2186-11-11 16:45:00,2186-11-12 08:27:00,0,1,0,0,"K743,A419,J189",NaT,NaN,None
0,M316,Other giant cell arteritis,19997911,20274882,7,"rare_circulatory_system_diseases, rare_renal_d...",True,3,F,79,2188,2008 - 2010,NaN,2196-08-08 05:44:00,2196-08-10 15:30:00,NaN,OBSERVATION ADMIT,EMERGENCY ROOM,HOME,Medicare,ENGLISH,MARRIED,WHITE - OTHER EUROPEAN,2196-08-08 02:05:00,2196-08-08 07:45:00,0,1,2,0,"M316,N179",2196-09-16 23:53:00,37.000,None
0,Q453,Other congenital malformations of pancreas and...,19999464,23033573,2,Uncategorised,True,5,F,35,2160,2008 - 2010,NaN,2171-08-01 18:34:00,2171-08-04 16:20:00,NaN,OBSERVATION ADMIT,WALK-IN/SELF REFERRAL,HOME,Other,ENGLISH,MARRIED,WHITE,2171-08-01 15:12:00,2171-08-01 19:31:00,0,1,0,0,Q453,NaT,NaN,None
0,I10,Essential (primary) hypertension,19999466,21397174,2,common_diseases,True,4,M,33,2116,2017 - 2019,NaN,2116-08-30 15:21:00,2116-08-31 21:49:00,NaN,DIRECT OBSERVATION,CLINIC REFERRAL,NaN,Medicare,ENGLISH,NaN,OTHER,2116-08-30 12:11:00,2116-08-31 21:49:00,0,1,0,0,I10,NaT,NaN,None
0,G960,Cerebrospinal fluid leak,19999784,23664472,4,rare_neurological_diseases,True,3,M,57,2119,2017 - 2019,NaN,2119-07-24 03:59:00,2119-08-09 17:56:00,NaN,ELECTIVE,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,NaN,NaN,0,1,9,1,G960,2119-08-11 11:36:00,1.000,5
0,I10,Essential (primary) hypertension,19999828,29734428,9,common_diseases,True,4,F,46,2147,2017 - 2019,NaN,2147-07-18 16:23:00,2147-08-04 18:10:00,NaN,EW EMER.,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2147-07-17 17:18:00,2147-07-18 17:34:00,0,1,1,0,"I10,J449,E119",2149-01-08 16:44:00,522.000,None


In [ ]:
#merged_admissions[merged_admissions['subject_id']==13982153]
merged_admissions[merged_admissions['subject_id']==19997911]

,icd_code,long_title,subject_id,hadm_id,seq_num,Classification,included,class,gender,anchor_age,anchor_year,anchor_year_group,dod,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,is_first_admission,num_followups,future_rare_skin,icd_code_list,next_admittime,days_to_next_admission,time_to_diagnosis
0,M316,Other giant cell arteritis,19997911,20274882,7,"rare_circulatory_system_diseases, rare_renal_d...",True,3,F,79,2188,2008 - 2010,NaN,2196-08-08 05:44:00,2196-08-10 15:30:00,NaN,OBSERVATION ADMIT,EMERGENCY ROOM,HOME,Medicare,ENGLISH,MARRIED,WHITE - OTHER EUROPEAN,2196-08-08 02:05:00,2196-08-08 07:45:00,0,1,2,0,"M316,N179",2196-09-16 23:53:00,37.000,None


In [ ]:
merged_adm_pat_sorted[merged_adm_pat_sorted['subject_id']==11528828]

,icd_code,long_title,subject_id,hadm_id,seq_num,Classification,included,class,gender,anchor_age,anchor_year,anchor_year_group,dod,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,is_first_admission,num_followups,future_rare_skin,icd_code_list,next_admittime,days_to_next_admission,time_to_diagnosis
0,Q211,Atrial septal defect,11528828,24389063,2,Uncategorised,True,5,M,73,2159,2008 - 2010,2170-03-10,2169-03-28 19:31:00,2169-03-31 16:00:00,NaN,OBSERVATION ADMIT,PACU,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,0,1,5,1,Q211,2169-09-21 14:17:00,173.000,2
1,B372,Candidiasis of skin and nail,11528828,25610293,22,"rare_genetic_diseases, rare_immunological_dise...",True,1,M,73,2159,2008 - 2010,2170-03-10,2169-09-21 14:17:00,2169-10-04 13:35:00,NaN,OBSERVATION ADMIT,CLINIC REFERRAL,AGAINST ADVICE,Medicare,ENGLISH,MARRIED,WHITE,2169-09-21 11:14:00,2169-09-21 15:56:00,0,0,4,0,"B372,I471,I5033,N179,Q211",2169-10-05 00:23:00,0.000,2
2,I471,Supraventricular tachycardia,11528828,25610293,4,"rare_cardiac_diseases, rare_genetic_diseases",True,3,M,73,2159,2008 - 2010,2170-03-10,2169-09-21 14:17:00,2169-10-04 13:35:00,NaN,OBSERVATION ADMIT,CLINIC REFERRAL,AGAINST ADVICE,Medicare,ENGLISH,MARRIED,WHITE,2169-09-21 11:14:00,2169-09-21 15:56:00,0,0,4,0,"B372,I471,I5033,N179,Q211",2169-10-05 00:23:00,0.000,2
3,I5033,Acute on chronic diastolic (congestive) heart ...,11528828,25610293,2,common_diseases,True,4,M,73,2159,2008 - 2010,2170-03-10,2169-09-21 14:17:00,2169-10-04 13:35:00,NaN,OBSERVATION ADMIT,CLINIC REFERRAL,AGAINST ADVICE,Medicare,ENGLISH,MARRIED,WHITE,2169-09-21 11:14:00,2169-09-21 15:56:00,0,0,4,0,"B372,I471,I5033,N179,Q211",2169-10-05 00:23:00,0.000,2
4,N179,"Acute kidney failure, unspecified",11528828,25610293,7,common_diseases,True,4,M,73,2159,2008 - 2010,2170-03-10,2169-09-21 14:17:00,2169-10-04 13:35:00,NaN,OBSERVATION ADMIT,CLINIC REFERRAL,AGAINST ADVICE,Medicare,ENGLISH,MARRIED,WHITE,2169-09-21 11:14:00,2169-09-21 15:56:00,0,0,4,0,"B372,I471,I5033,N179,Q211",2169-10-05 00:23:00,0.000,2
5,Q211,Atrial septal defect,11528828,25610293,6,Uncategorised,True,5,M,73,2159,2008 - 2010,2170-03-10,2169-09-21 14:17:00,2169-10-04 13:35:00,NaN,OBSERVATION ADMIT,CLINIC REFERRAL,AGAINST ADVICE,Medicare,ENGLISH,MARRIED,WHITE,2169-09-21 11:14:00,2169-09-21 15:56:00,0,0,4,0,"B372,I471,I5033,N179,Q211",2169-10-05 00:23:00,0.000,2
6,B372,Candidiasis of skin and nail,11528828,22827232,23,"rare_genetic_diseases, rare_immunological_dise...",True,1,M,73,2159,2008 - 2010,2170-03-10,2169-10-05 00:23:00,2169-10-12 10:30:00,NaN,OBSERVATION ADMIT,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,2169-10-04 16:58:00,2169-10-05 01:51:00,0,0,3,0,"B372,I5033,N179",2169-11-22 15:14:00,41.000,2
7,I5033,Acute on chronic diastolic (congestive) heart ...,11528828,22827232,2,common_diseases,True,4,M,73,2159,2008 - 2010,2170-03-10,2169-10-05 00:23:00,2169-10-12 10:30:00,NaN,OBSERVATION ADMIT,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,2169-10-04 16:58:00,2169-10-05 01:51:00,0,0,3,0,"B372,I5033,N179",2169-11-22 15:14:00,41.000,2
8,N179,"Acute kidney failure, unspecified",11528828,22827232,8,common_diseases,True,4,M,73,2159,2008 - 2010,2170-03-10,2169-10-05 00:23:00,2169-10-12 10:30:00,NaN,OBSERVATION ADMIT,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,2169-10-04 16:58:00,2169-10-05 01:51:00,0,0,3,0,"B372,I5033,N179",2169-11-22 15:14:00,41.000,2
9,I471,Supraventricular tachycardia,11528828,23389059,8,"rare_cardiac_diseases, rare_genetic_diseases",True,3,M,73,2159,2008 - 2010,2170-03-10,2169-11-22 15:14:00,2169-12-05 15:03:00,NaN,EW EMER.,PHYSICIAN REFERRAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,2169-11-21 22:38:00,2169-11-22 17:04:00,0,0,2,0,"I471,I5033,N179",2169-12-11 22:07:00,6.000,2


In [ ]:
#Checking if time to diagnosis column is working correctly or not.
# Filter merged_admissions for patients who will be diagnosed with rare skin disease in the future
future_diag_df = merged_admissions[merged_admissions['future_rare_skin'] == 1]

# Select relevant columns to inspect
cols_to_view = ['subject_id', 'hadm_id', 'admittime', 'dischtime',
                'time_to_diagnosis', 'future_rare_skin', 'class', 'num_followups']

# Display the filtered DataFrame
future_diag_df[cols_to_view].sort_values(by='time_to_diagnosis').head(20)

,subject_id,hadm_id,admittime,dischtime,time_to_diagnosis,future_rare_skin,class,num_followups
0,11528828,24389063,2169-03-28 19:31:00,2169-03-31 16:00:00,2,1,5,5
0,11388341,27040288,2143-08-17 12:23:00,2143-09-29 10:20:00,2,1,2,1
0,18137182,29270173,2136-05-16 17:40:00,2136-05-18 17:19:00,2,1,3,1
0,13885968,21196228,2147-07-08 17:23:00,2147-07-11 17:44:00,2,1,4,3
0,11578849,29988659,2182-10-26 00:27:00,2182-10-29 15:06:00,2,1,4,1
0,13825646,29303703,2121-04-17 04:59:00,2121-04-25 18:00:00,2,1,3,4
0,15195372,24101989,2117-07-14 17:10:00,2117-07-15 16:15:00,2,1,3,1
0,11638303,28795143,2146-01-23 07:26:00,2146-01-24 16:25:00,2,1,3,2
0,11347834,26031822,2165-10-09 01:07:00,2165-10-11 16:59:00,2,1,4,4
0,17859352,21311255,2161-01-08 18:08:00,2161-02-26 18:20:00,2,1,4,2


In [ ]:
merged_admissions['time_to_diagnosis'].value_counts()

,count
time_to_diagnosis,
1,1415
3,130
2,106
4,102
5,80
6,51
7,41
12,33
9,32


In [ ]:
# write the cleaned data frame to a new Excel file
merged_adm_pat_sorted.to_excel("/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Data/merged_adm_pat_sorted.xlsx", index=False, header = True)
merged_admissions.to_excel("/content/drive/MyDrive/ColabNotebooks/Rare_Diseases_Research/Data/all_disorders_details.xlsx", index=False, header = True)